<a href="https://colab.research.google.com/github/LuisAL79/TCCs/blob/main/TCC_GPS_MAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bibliotecas**

In [ ]:
!pip install skyfield opencage pymap3d

# **Mostrar Coordenadas $(x,y,z)$ dos satélites visíveis em uma cidade**

In [ ]:
import numpy as np
from skyfield.api import load
from skyfield.toposlib import wgs84
from geopy.geocoders import Nominatim
import pandas as pd

def satelites_visiveis(nome_da_cidade, max_satelites=4, elevation=200):

    # Carregar os dados TLE dos satélites GPS
    satellites = load.tle_file('https://celestrak.com/NORAD/elements/gps-ops.txt')
    print(f'Carregados {len(satellites)} satélites.')

    # Obter a localização da cidade usando geopy
    geolocator = Nominatim(user_agent="satellite_orbit_visualization")
    location = geolocator.geocode(nome_da_cidade)
    if location is None:
        raise ValueError(f"Não foi possível encontrar a localização da cidade: {nome_da_cidade}")
    latitude, longitude = location.latitude, location.longitude
    print(latitude, longitude)

    # Criar o observador na cidade especificada
    ts = load.timescale()
    time_now = ts.now()
    observer = wgs84.latlon(latitude, longitude, elevation)

    # Velocidade da luz em km/s
    speed_of_light_km_s = 299792.458

    # Lista para armazenar os dados dos satélites visíveis
    satellite_data = []

    for sat in satellites:
        difference = sat - observer
        topocentric = difference.at(time_now)
        alt, az, distance = topocentric.altaz()

        if alt.degrees > 15:  # Apenas satélites acima do horizonte
            geocentric = sat.at(time_now).position.km  # Coordenadas espaciais [X, Y, Z]
            response_time = distance.km / speed_of_light_km_s  # Tempo de resposta (s)

            satellite_data.append({
                'Satélite': sat.name,
                'X (km)': geocentric[0],
                'Y (km)': geocentric[1],
                'Z (km)': geocentric[2],
                'Distância (km)': distance.km,
            })

            # Parar quando atingir o número máximo de satélites definidos
            if len(satellite_data) >= max_satelites:
                break

    # Criar um DataFrame com os dados dos satélites visíveis
    df = pd.DataFrame(satellite_data)
    return df

cidade = "Santana do Araguaia, Pará, Brasil"
df = satelites_visiveis(nome_da_cidade=cidade, max_satelites=4, elevation=200)
df

Carregados 31 satélites.
-9.3348334 -50.3427734


,Satélite,X (km),Y (km),Z (km),Distância (km)
0,GPS BIIR-8 (PRN 16),16657.664536,20975.531476,648.751757,23570.146738
1,GPS BIIR-13 (PRN 02),23830.294972,1076.574332,11653.293302,21685.990726
2,GPS BIIRM-6 (PRN 07),20139.022006,-12920.149826,11595.995040,22699.183183
3,GPS BIIF-7 (PRN 09),14568.856636,-18104.195229,-12892.544371,23069.525835


# **Função de Trilateração 1**

In [ ]:
from sympy import *

def resolver_sistema_trilateracao(sat_df):

  # Selecionar apenas as 4 primeiras linhas
  sat_df = sat_df.iloc[:4]

  # Definir as variáveis simbólicas
  x, y, z = symbols('x y z')

  # Extrair pontos e distâncias do DataFrame
  pontos = sat_df[['X (km)', 'Y (km)', 'Z (km)']].to_numpy()
  distancias = sat_df['Distância (km)'].to_numpy()

  # Criar as equações baseadas nas distâncias e coordenadas
  equacoes = []
  for (xs, ys, zs), ds in zip(pontos,distancias):
      equacao = (x - xs)**2 + (y - ys)**2 + (z - zs)**2 - ds**2
      equacoes.append(Eq(equacao.expand(), 0))

  # Subtrair as equações 1, 2, 3 da primeira
  equacoes_subtraidas = []
  primeira_equacao = equacoes[0]
  for i in range(1, len(equacoes)):
    # Subtrair os lados esquerdos e direitos das equações
    lhs_subtraido = equacoes[i].lhs - primeira_equacao.lhs
    rhs_subtraido = equacoes[i].rhs - primeira_equacao.rhs
    # Criar a nova equação após a subtração
    equacao_subtraida = Eq(lhs_subtraido, rhs_subtraido)
    equacoes_subtraidas.append(nsimplify(equacao_subtraida, tolerance=1e-9).evalf())

  # Exibir as equações subtraídas
  for eq in equacoes_subtraidas:
    display(eq)

  # Resolver o sistema de equações não lineares
  solucoes = solve(equacoes_subtraidas, (x, y, z))
  sols = [solucoes[var] for var in (x, y, z)]

  # Converter os resultados para float numérico (numpy)
  sols = np.array([float(sol.evalf()) for sol in sols])
  X, Y, Z = sols[0], sols[1], sols[2]

  return X, Y, Z

# **Função para transformar em coordendas geográficas**

In [ ]:
def trans_geographic(X,Y,Z):

  R = 6378.1370  # (km) - o raio no equador

  r = np.sqrt(X**2 + Y**2 + Z**2) ; rho = np.sqrt(X**2 + Y**2)

  # Calcular a latitude (theta) e longitude (phi)
  theta = np.degrees(np.arcsin(Z/r))
  phi = np.degrees(np.arcsin(Y/rho))
  alt = r - R

  print(f'Coordenadas espaciais -> X: {X} km, Y: {Y} km, Z: {Z} km')
  print(f'Coordenadas geográficas -> latitude: {theta}, longitude: {phi}, altitude: {alt}')

  return theta, phi, alt

# **Função para exibir a localização no mapa**

In [ ]:
import folium
from folium import plugins

def criar_mapa(loc, nome, raio=34, zoom=16):
  # Cria o mapa
  mapa = folium.Map(
      location=loc, zoom_start=zoom, control_scale=True
  )

  # Background
  folium.TileLayer(tiles= 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
      attr = 'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',
      name = 'Esri.WorldImagery'
  ).add_to(mapa)

  folium.LayerControl().add_to(mapa)

  # Adiciona o ícone personalizado, se fornecido
  figura = "https://ederepente50.wordpress.com/wp-content/uploads/2020/12/snoopy-2.jpg"
  if figura:
      icon = folium.features.CustomIcon(figura, icon_size=(50, 50))
  else:
      icon = folium.Icon(color='blue')
  icon1 = 'glyphicon glyphicon-send'

  # Adiciona o marcador com a imagem personalizada
  folium.Marker(location=loc, popup=nome, icon=icon
                ).add_to(mapa)

  # Adiciona o círculo marcador
  folium.CircleMarker(
      location=loc, popup=nome, radius=raio, color='red', fill=True, fill_color='blue'
  ).add_to(mapa)

  # plugins
  mapa.add_child(folium.LatLngPopup())

  mapa.save('Meu_mapinha.html')

  return mapa

# **Função principal para calcular posição do receptor**

In [ ]:
# Calcular posição do observador
def fun_posfinal(nome_da_cidade, max_satelites=4, elevation=200):
  df = satelites_visiveis(nome_da_cidade, max_satelites, elevation)
  X, Y, Z = resolver_sistema_trilateracao(df)
  theta, phi, _ = trans_geographic(X, Y, Z)
  display(df)

  return theta, phi

nome_da_cidade = "Santana do Araguaia, Pará, Brasil"
altitude = 200
lati,longi = fun_posfinal(nome_da_cidade, max_satelites=4, elevation=altitude)

Carregados 31 satélites.
-9.3348334 -50.3427734


Eq(-14345.8182947102*x + 39796.1058895036*y - 22020.527075008*z + 72212890.4043558, 0)

Eq(-6970.58678578612*x + 67791.4151556899*y - 21895.3571047223*z + 29423197.1647189, 0)

Eq(4172.26868985388*x + 78157.0758935738*y + 27081.481567668*z + 11705504.7291188, 0)

Coordenadas espaciais -> X: 6290.662453115285 km, Y: -124.12164776974103 km, Z: -1043.1797686299449 km
Coordenadas geográficas -> latitude: -9.413867561895655, longitude: -1.1303616970758352, altitude: -0.3580517956706899


,Satélite,X (km),Y (km),Z (km),Distância (km)
0,GPS BIIR-8 (PRN 16),16656.048528,20976.964774,645.687083,23570.094355
1,GPS BIIR-13 (PRN 02),23828.957676,1078.911829,11655.950621,21686.562694
2,GPS BIIRM-6 (PRN 07),20141.341921,-12918.742804,11593.365636,22698.588767
3,GPS BIIF-7 (PRN 09),14569.914183,-18101.573173,-12895.053701,23069.496922


In [ ]:
loc= [lati, longi]
#loc = [-9.361061790808126,-50.345490494971116]
mapa = criar_mapa(loc, nome='Santana do Araguaia',zoom=12)
mapa